# Model Training

1.1 Import Data and Required packages

In [56]:
# Basic Import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [57]:
# Modelling
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings('ignore')


Imprt the CSV data as Pandas dataframe

In [58]:
df = pd.read_csv("data/stud.csv")

In [59]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


Split into x and y variable

In [60]:
x =df.drop(columns=['math_score'],axis=1)

In [61]:
x.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [62]:
y = df['math_score']
y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math_score, Length: 1000, dtype: int64

In [63]:
numeric_features = [i for i in df.columns if df[i].dtype != 'O']
categorical_features = [i for i in df.columns if df[i].dtype == 'O']


In [64]:
for i in categorical_features:
    print("Categories in",i,x[i].unique())

Categories in gender ['female' 'male']
Categories in race_ethnicity ['group B' 'group C' 'group A' 'group D' 'group E']
Categories in parental_level_of_education ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
Categories in lunch ['standard' 'free/reduced']
Categories in test_preparation_course ['none' 'completed']


In [65]:
# Create Column Transformer with 3 types of transformers
num_features = x.select_dtypes(exclude="object").columns
cat_features = x.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

In [66]:
x= preprocessor.fit_transform(x)

In [67]:
x.shape

(1000, 19)

In [68]:
#split into train and test data
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.20,random_state=42)
x_train.shape ,x_test.shape 

((800, 19), (200, 19))

Create an Evaluate Function to give all metrics after model Training

In [69]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    rmse = np.sqrt(mean_squared_error(true,predicted))
    r2_square = r2_score(true,predicted)

    return mae,rmse,r2_score

In [70]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(x_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {}".format(model_train_rmse))
    print("- Mean Absolute Error: {}".format(model_train_mae))
    print("- R2 Score: {}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {}".format(model_test_rmse))
    print("- Mean Absolute Error: {}".format(model_test_mae))
    print("- R2 Score: {}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 5.325499939172572
- Mean Absolute Error: 4.26845703125
- R2 Score: <function r2_score at 0x000002B0A7142EE0>
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.4008924965290435
- Mean Absolute Error: 4.218388671875
- R2 Score: <function r2_score at 0x000002B0A7142EE0>


Lasso
Model performance for Training set
- Root Mean Squared Error: 6.593815587795565
- Mean Absolute Error: 5.206302661246526
- R2 Score: <function r2_score at 0x000002B0A7142EE0>
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 6.519694535667419
- Mean Absolute Error: 5.157881810347763
- R2 Score: <function r2_score at 0x000002B0A7142EE0>


Ridge
Model performance for Training set
- Root Mean Squared Error: 5.323324922741654
- Mean Absolute Error: 4.26498782372598
- R2 Score: <function r2_score at 0x000002B0A7142EE0>
----------------------------------
M

# Results

In [71]:
pd.DataFrame(list(zip(model_list,r2_list)),columns=['Model Name','R2_score'])

,Model Name,R2_score
0,Linear Regression,<function r2_score at 0x000002B0A7142EE0>
1,Lasso,<function r2_score at 0x000002B0A7142EE0>
2,Ridge,<function r2_score at 0x000002B0A7142EE0>
3,K-Neighbors Regressor,<function r2_score at 0x000002B0A7142EE0>
4,Decision Tree,<function r2_score at 0x000002B0A7142EE0>
5,Random Forest Regressor,<function r2_score at 0x000002B0A7142EE0>
6,XGBRegressor,<function r2_score at 0x000002B0A7142EE0>
7,CatBoosting Regressor,<function r2_score at 0x000002B0A7142EE0>
8,AdaBoost Regressor,<function r2_score at 0x000002B0A7142EE0>
